In [1]:
# author: Yin 

# 4 27 2020, change panel identifier from id to csid, get identical results as LG

# Three alternatives: sq,a,b; 
# arc elas are not tested: towndown townmid townup (dummy)
# point elas: other variables
# 04 04 full dataset
# 04 16 test blocks

In [2]:
import sys
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.results as res
from biogeme.expressions import *

In [3]:
print("Running 01logit_elas_dir_0427.py...")

Running 01logit_elas_dir_0427.py...


In [4]:
# Read the data
df = pd.read_csv("agree0623_all_0427.csv")
database = db.Database("agree0623_all_0427",df)
# They are organized as panel data. The variable ID identifies each individual.
database.panel("csid")
# use the names of the variable as Python variable.
globals().update(database.variables)

In [5]:
#exclude=(block1==0)
#exclude=(block1==1)

#database.remove(exclude)

In [6]:
# Normalize the weights
sumWeight = database.data['weight'].sum()
print(sumWeight) 
##result=7423.735097, sample size=7414

7423.735097


In [7]:
# 3717.76002885, block1 sample size=3713
##result=3705.97506815, block2 sample size=3701
#normalizedWeight = weight * 3713 / 3717.76002885
normalizedWeight = weight * 7414 / 7423.735097
# originally weight the whole sample use 1123 cases, 
# then delete nonresponse for choice questions, and 945 cases are used for logit

In [8]:
# Parameters to be estimated, only for continuous variables
#NZone variables are already scaled: zone/100
#use lg estimates as starting values 
B_NZONE1 = Beta('B_NZONE1',-0.0682,None,None,0)
B_NZONE2 = Beta('B_NZONE2',-0.0372,None,None,0)
B_NZONE3 = Beta('B_NZONE3',-0.0283,None,None,0)
B_TOWNDOWN = Beta('B_TOWNDOWN',0.0274,None,None,0)
B_TOWNMID = Beta('B_TOWNMID',0.11435,None,None,0)
B_TOWNUP = Beta('B_TOWNUP',0.205,None,None,0)
B_ECO = Beta('B_ECO',0.053,None,None,0)
B_REC = Beta('B_REC',0.0089,None,None,0)
B_DRY = Beta('B_DRY',-0.2146,None,None,0)
B_TAX = Beta('B_TAX',-0.0257,None,None,0)


In [9]:
# Definition of the utility functions
VSQ = B_NZONE1 * nzone1_sq + B_NZONE2 * nzone2_sq + B_NZONE3 * nzone3_sq + B_TOWNDOWN * towndown_sq + B_TOWNMID * townmid_sq + B_TOWNUP * townup_sq + B_ECO * eco_sq + B_REC * rec_sq + B_DRY * dry_sq + B_TAX * tax_sq
VA =  B_NZONE1 * nzone1_a + B_NZONE2 * nzone2_a + B_NZONE3 * nzone3_a + B_TOWNDOWN * towndown_a + B_TOWNMID * townmid_a + B_TOWNUP * townup_a + B_ECO * eco_a + B_REC * rec_a + B_DRY * dry_a + B_TAX * tax_a
VB =  B_NZONE1 * nzone1_b + B_NZONE2 * nzone2_b + B_NZONE3 * nzone3_b + B_TOWNDOWN * towndown_b + B_TOWNMID * townmid_b + B_TOWNUP * townup_b + B_ECO * eco_b + B_REC * rec_b + B_DRY * dry_b + B_TAX * tax_b

In [10]:
# Associate utility functions with the numbering of alternatives
V = {1: VSQ,
     2: VA,
     3: VB}

# Associate the availability conditions with the alternatives
# all alternatives are available for each individual
av = {1: 1,
      2: 1,
      3: 1}

In [11]:
# Definition of the model. This is the contribution of each observation to the log likelihood function.
prob_sq = models.logit(V,av,1)
prob_a  = models.logit(V,av,2)
prob_b  = models.logit(V,av,3)

In [12]:
# Calculation of the direct elasticities. 
# We use the "Derive" operator to calculate the derivatives. 
# alt_sq
direct_elas_sq_nzone1_sq = Derive(prob_sq,'nzone1_sq') * nzone1_sq / prob_sq 
direct_elas_sq_nzone2_sq = Derive(prob_sq,'nzone2_sq') * nzone2_sq / prob_sq 
direct_elas_sq_nzone3_sq = Derive(prob_sq,'nzone3_sq') * nzone3_sq / prob_sq 

direct_elas_sq_eco_sq = Derive(prob_sq,'eco_sq') * eco_sq / prob_sq 
direct_elas_sq_rec_sq = Derive(prob_sq,'rec_sq') * rec_sq / prob_sq 
direct_elas_sq_dry_sq = Derive(prob_sq,'dry_sq') * dry_sq / prob_sq 
direct_elas_sq_tax_sq = Derive(prob_sq,'tax_sq') * tax_sq / prob_sq 

# alt_a
direct_elas_a_nzone1_a = Derive(prob_a,'nzone1_a') * nzone1_a / prob_a 
direct_elas_a_nzone2_a = Derive(prob_a,'nzone2_a') * nzone2_a / prob_a 
direct_elas_a_nzone3_a = Derive(prob_a,'nzone3_a') * nzone3_a / prob_a 

direct_elas_a_eco_a = Derive(prob_a,'eco_a') * eco_a / prob_a 
direct_elas_a_rec_a = Derive(prob_a,'rec_a') * rec_a / prob_a 
direct_elas_a_dry_a = Derive(prob_a,'dry_a') * dry_a / prob_a 
direct_elas_a_tax_a = Derive(prob_a,'tax_a') * tax_a / prob_a

# alt_b
direct_elas_b_nzone1_b = Derive(prob_b,'nzone1_b') * nzone1_b / prob_b 
direct_elas_b_nzone2_b = Derive(prob_b,'nzone2_b') * nzone2_b / prob_b 
direct_elas_b_nzone3_b = Derive(prob_b,'nzone3_b') * nzone3_b / prob_b 

direct_elas_b_eco_b = Derive(prob_b,'eco_b') * eco_b / prob_b 
direct_elas_b_rec_b = Derive(prob_b,'rec_b') * rec_b / prob_b 
direct_elas_b_dry_b = Derive(prob_b,'dry_b') * dry_b / prob_b 
direct_elas_b_tax_b = Derive(prob_b,'tax_b') * tax_b / prob_b


In [13]:
simulate = {'nweight': normalizedWeight,
            'Prob. No Action': prob_sq,
            'Prob. Plan A': prob_a,
            'Prob. Plan B':prob_b,
            'direct_elas_sq_nzone1_sq':direct_elas_sq_nzone1_sq,
            'direct_elas_sq_nzone2_sq':direct_elas_sq_nzone2_sq,
            'direct_elas_sq_nzone3_sq':direct_elas_sq_nzone3_sq,
            'direct_elas_sq_eco_sq':direct_elas_sq_eco_sq,
            'direct_elas_sq_rec_sq':direct_elas_sq_rec_sq,
            'direct_elas_sq_dry_sq':direct_elas_sq_dry_sq,
            'direct_elas_sq_tax_sq':direct_elas_sq_tax_sq,
            'direct_elas_a_nzone1_a':direct_elas_a_nzone1_a,
            'direct_elas_a_nzone2_a':direct_elas_a_nzone2_a,
            'direct_elas_a_nzone3_a':direct_elas_a_nzone3_a,
            'direct_elas_a_eco_a':direct_elas_a_eco_a,
            'direct_elas_a_rec_a':direct_elas_a_rec_a,
            'direct_elas_a_dry_a':direct_elas_a_dry_a,
            'direct_elas_a_tax_a':direct_elas_a_tax_a,
            'direct_elas_b_nzone1_b':direct_elas_b_nzone1_b,
            'direct_elas_b_nzone2_b':direct_elas_b_nzone2_b,
            'direct_elas_b_nzone3_b':direct_elas_b_nzone3_b,
            'direct_elas_b_eco_b':direct_elas_b_eco_b,
            'direct_elas_b_rec_b':direct_elas_b_rec_b,
            'direct_elas_b_dry_b':direct_elas_b_dry_b,
            'direct_elas_b_tax_b':direct_elas_b_tax_b}


In [14]:
biogeme  = bio.BIOGEME(database,simulate)
biogeme.modelName = "01logit_elas_dir_pt_0427"

In [15]:
# Retrieve the values of the parameters
# First, extract the names of parameters needed for the simulation
betas = biogeme.freeBetaNames

# Read the estimation results from the file
results = res.bioResults(pickleFile='01logit_0427.pickle')

# Extract the values that are necessary
betaValues = results.getBetaValues(betas)

# simulatedValues is a Panda dataframe with the same number of rows as the database, and as many columns as formulas to simulate.
# simulate the formulas using the nomial values
simulatedValues = biogeme.simulate(betaValues)

In [16]:
# We calculate the weighted probabilitie
simulatedValues['Weighted Prob. No Action'] = simulatedValues['nweight'] * simulatedValues['Prob. No Action']

simulatedValues['Weighted Prob. Plan A'] = simulatedValues['nweight'] * simulatedValues['Prob. Plan A']

simulatedValues['Weighted Prob. Plan B'] = simulatedValues['nweight'] * simulatedValues['Prob. Plan B']


In [17]:
denominator_sq = simulatedValues['Weighted Prob. No Action'].sum()
denominator_a  = simulatedValues['Weighted Prob. Plan A'].sum()
denominator_b  = simulatedValues['Weighted Prob. Plan B'].sum()

In [18]:
# aggregate elasticities_sq
# function=sum(E*Weighted prob/sum(weighted prob))
direct_elas_term_sq_nzone1_sq = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['direct_elas_sq_nzone1_sq'] / denominator_sq).sum()
print(f"Aggregate direct elasticity of sq wrt nzone1_sq: {direct_elas_term_sq_nzone1_sq:.3g}")
direct_elas_term_sq_nzone2_sq = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['direct_elas_sq_nzone2_sq'] / denominator_sq).sum()
print(f"Aggregate direct elasticity of sq wrt nzone2_sq: {direct_elas_term_sq_nzone2_sq:.3g}")
direct_elas_term_sq_nzone3_sq = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['direct_elas_sq_nzone3_sq'] / denominator_sq).sum()
print(f"Aggregate direct elasticity of sq wrt nzone3_sq: {direct_elas_term_sq_nzone3_sq:.3g}")

direct_elas_term_sq_eco_sq = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['direct_elas_sq_eco_sq'] / denominator_sq).sum()
print(f"Aggregate direct elasticity of sq wrt eco_sq: {direct_elas_term_sq_eco_sq:.3g}")
direct_elas_term_sq_rec_sq = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['direct_elas_sq_rec_sq'] / denominator_sq).sum()
print(f"Aggregate direct elasticity of sq wrt rec_sq: {direct_elas_term_sq_rec_sq:.3g}")
direct_elas_term_sq_dry_sq = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['direct_elas_sq_dry_sq'] / denominator_sq).sum()
print(f"Aggregate direct elasticity of sq wrt dry_sq: {direct_elas_term_sq_dry_sq:.3g}")

direct_elas_term_sq_tax_sq = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['direct_elas_sq_tax_sq'] / denominator_sq).sum()
print(f"Aggregate direct elasticity of sq wrt tax_sq: {direct_elas_term_sq_tax_sq:.3g}")


# aggregate elasticities_a
direct_elas_term_a_nzone1_a = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['direct_elas_a_nzone1_a'] / denominator_a).sum()
print(f"Aggregate direct elasticity of a wrt nzone1_a: {direct_elas_term_a_nzone1_a:.3g}")
direct_elas_term_a_nzone2_a = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['direct_elas_a_nzone2_a'] / denominator_a).sum()
print(f"Aggregate direct elasticity of a wrt nzone2_a: {direct_elas_term_a_nzone2_a:.3g}")
direct_elas_term_a_nzone3_a = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['direct_elas_a_nzone3_a'] / denominator_a).sum()
print(f"Aggregate direct elasticity of a wrt nzone3_a: {direct_elas_term_a_nzone3_a:.3g}")

direct_elas_term_a_eco_a = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['direct_elas_a_eco_a'] / denominator_a).sum()
print(f"Aggregate direct elasticity of a wrt eco_a: {direct_elas_term_a_eco_a:.3g}")
direct_elas_term_a_rec_a = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['direct_elas_a_rec_a'] / denominator_a).sum()
print(f"Aggregate direct elasticity of a wrt rec_a: {direct_elas_term_a_rec_a:.3g}")
direct_elas_term_a_dry_a = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['direct_elas_a_dry_a'] / denominator_a).sum()
print(f"Aggregate direct elasticity of a wrt dry_a: {direct_elas_term_a_dry_a:.3g}")

direct_elas_term_a_tax_a = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['direct_elas_a_tax_a'] / denominator_a).sum()
print(f"Aggregate direct elasticity of a wrt tax_a: {direct_elas_term_a_tax_a:.3g}")


# aggregate elasticities_b
direct_elas_term_b_nzone1_b = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['direct_elas_b_nzone1_b'] / denominator_b).sum()
print(f"Aggregate direct elasticity of b wrt nzone1_b: {direct_elas_term_b_nzone1_b:.3g}")
direct_elas_term_b_nzone2_b = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['direct_elas_b_nzone2_b'] / denominator_b).sum()
print(f"Aggregate direct elasticity of b wrt nzone2_b: {direct_elas_term_b_nzone2_b:.3g}")
direct_elas_term_b_nzone3_b = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['direct_elas_b_nzone3_b'] / denominator_b).sum()
print(f"Aggregate direct elasticity of b wrt nzone3_b: {direct_elas_term_b_nzone3_b:.3g}")

direct_elas_term_b_eco_b = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['direct_elas_b_eco_b'] / denominator_b).sum()
print(f"Aggregate direct elasticity of b wrt eco_b: {direct_elas_term_b_eco_b:.3g}")
direct_elas_term_b_rec_b = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['direct_elas_b_rec_b'] / denominator_b).sum()
print(f"Aggregate direct elasticity of b wrt rec_b: {direct_elas_term_b_rec_b:.3g}")
direct_elas_term_b_dry_b = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['direct_elas_b_dry_b'] / denominator_b).sum()
print(f"Aggregate direct elasticity of b wrt dry_b: {direct_elas_term_b_dry_b:.3g}")

direct_elas_term_b_tax_b = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['direct_elas_b_tax_b'] / denominator_b).sum()
print(f"Aggregate direct elasticity of b wrt tax_b: {direct_elas_term_b_tax_b:.3g}")



Aggregate direct elasticity of sq wrt nzone1_sq: -0.747
Aggregate direct elasticity of sq wrt nzone2_sq: -0.363
Aggregate direct elasticity of sq wrt nzone3_sq: -0.424
Aggregate direct elasticity of sq wrt eco_sq: 1.18
Aggregate direct elasticity of sq wrt rec_sq: 0.284
Aggregate direct elasticity of sq wrt dry_sq: -0.34
Aggregate direct elasticity of sq wrt tax_sq: 0
Aggregate direct elasticity of a wrt nzone1_a: -0.349
Aggregate direct elasticity of a wrt nzone2_a: -0.249
Aggregate direct elasticity of a wrt nzone3_a: -0.322
Aggregate direct elasticity of a wrt eco_a: 1.02
Aggregate direct elasticity of a wrt rec_a: 0.346
Aggregate direct elasticity of a wrt dry_a: -0.167
Aggregate direct elasticity of a wrt tax_a: -0.226
Aggregate direct elasticity of b wrt nzone1_b: -0.412
Aggregate direct elasticity of b wrt nzone2_b: -0.266
Aggregate direct elasticity of b wrt nzone3_b: -0.34
Aggregate direct elasticity of b wrt eco_b: 1.1
Aggregate direct elasticity of b wrt rec_b: 0.318
Aggrega